# First Iteration
Now that I am done scraping (please dear god no more I dont want to for the time being), I'm going to do a clustering thingy thing!

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import numpy as np
import nltk
import os
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, TruncatedSVD

from sklearn.model_selection import train_test_split

from sklearn.neighbors import NearestNeighbors

/Users/jennwon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/jennwon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/jennwon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/jennwon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/jennwon/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/jennwon/anaconda3/lib/python

In [6]:
from glob import glob

path_names = glob('Lyrics_folder_818/*.pkl')
len(path_names)

17226

In [19]:
song_df = pd.DataFrame()
for path in path_names[:]:
    df = pd.DataFrame.from_dict([pd.read_pickle(str(path))])
    
    song_df = song_df.append(df)
    
song_df

,artist,lyrics,song
0,The Cascades,The last leaf clings to the bough\nJust one le...,The Last Leaf
0,Keith Urban,[Verse 1]\nI woke up early this morning around...,Youll Think Of Me
0,BJ Thomas,Who's the one who tied you shoe when you were ...,Mama
0,Al Wilson,I've got a feeling\nThat we'll be seeing\nEach...,Ive Got A Feeling Well Be Seeing Each Other Again
0,The Pointer Sisters,"[Chant:] (4x)\nBetcha got a chick on the side,...",How Long Betcha Got A Chick On The Side
0,The Mixtures,Riding along my pushbike honey\nWhen I noticed...,Pushbike Song
0,Greg Kihn Band,Oh Oh Oh Oh\nWhere were you when I needed you?...,Jeopardy
0,Expose,More than ever you're on my mind\nBeing lonely...,I Wish The Phone Would Ring
0,The Brand New Heavies,"(Gotta do right, cha gotta do right, ahh somet...",Sometimes
0,Carpenters,[Verse]\nThe hardest thing I've ever done\nIs ...,I Need To Be In Love


In [20]:
# loading in my dataset

#song_df = pd.DataFrame(pd.read_pickle('third_round_results_lyrics.pkl')).reset_index()
#song_df.drop('index',axis = 1, inplace = True, errors = 'ignore')

train, test = train_test_split(song_df)
songs = train.copy(deep = True)
song = songs.drop(['artist', 'song'], axis = 1, errors = 'ignore')


song.head(5)

song_list = song.lyrics.tolist()

The interesting thing is that when you remove baby or like or love or honey, you only get 2 topics.... WHY

In [21]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [38]:
from sklearn.feature_extraction import text

my_stop_words = text.ENGLISH_STOP_WORDS.union(["na", 'instrumental', 'yeah', 'ha', 'ok', 'babe', 'oo', 'baby', 
                                              'doo', 'ooh', 'da', 'hey', 'la', 'oh', 'whoa', 'choo', 'thoia', 
                                              'thoing', 'ah', 'nigga', 'niggas', 'boom', 'woo', 'chorus'])

vectorizer = TfidfVectorizer(ngram_range=(1,1), stop_words=my_stop_words)

In [39]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2),  
                                   stop_words=my_stop_words, 
                                   token_pattern="\\b[a-z][a-z]+\\b",
                                   lowercase=True,
                                   max_df = 0.6)
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2),  
                                   stop_words=my_stop_words, 
                                   token_pattern="\\b[a-z][a-z]+\\b",
                                   lowercase=True,
                                   max_df = 0.6)

cv_data = count_vectorizer.fit_transform(song_list)
tfidf_data = tfidf_vectorizer.fit_transform(song_list)

#cv_data = count_vectorizer.fit_transform(test_list)
#tfidf_data = tfidf_vectorizer.fit_transform(list(lyrics.lyrics.values))

In [40]:
#im going to pickle things so that I dont have to fit again all the time because it takes so long

pd.to_pickle(cv_data, 'cv_data.pkl')
pd.to_pickle(tfidf_data, 'tfidf_data.pkl')

### From Pickles, I'm going to read in the fit transformed vectors

In [41]:
cv_data = pd.read_pickle('cv_data.pkl')
tfidf_data = pd.read_pickle('tfidf_data.pkl')

In [42]:
from sklearn.decomposition import NMF, TruncatedSVD

# IT TAKES 2 MINUTES FOR THIS TO RUN IM SO SAD

n_comp = 20
lsa_tfidf = TruncatedSVD(n_components=n_comp)
lsa_cv = TruncatedSVD(n_components=n_comp)
nmf_cv = NMF(n_components=n_comp)

lsa_tfidf_data = lsa_tfidf.fit_transform(tfidf_data)
lsa_cv_data = lsa_cv.fit_transform(cv_data)
nmf_cv_data = nmf_cv.fit_transform(cv_data)

In [43]:
display_topics(lsa_tfidf, tfidf_vectorizer.get_feature_names(),10)


Topic  0
love, don, know, ll, just, like, got, want, let, love love

Topic  1
love, love love, know love, ll love, need love, love don, love ll, love need, way love, true love

Topic  2
don, don know, know, want, don want, know don, don don, wanna, love don, don wanna

Topic  3
ll, heart, don, away, ll ll, forever, know, day, don know, time

Topic  4
come, come come, let, want, wanna, let let, dance, come let, tonight, need

Topic  5
want, need, girl, want want, don want, wanna, want love, need need, love want, ll

Topic  6
got, ve, come, ve got, come come, want, need, got got, don want, got love

Topic  7
let, dance, gonna, let let, wanna, dance dance, make, got, time, ve got

Topic  8
ll, got, ve got, gonna, know, ll ll, good, ve, need, ain

Topic  9
gonna, girl, little, wanna, ve got, ve, got, dance, little girl, little bit

Topic  10
let, girl, let let, know, little, ve, girl girl, got, little girl, let know

Topic  11
little, let, don, little bit, bit, love love, dance, just litt

In [44]:
display_topics(lsa_cv, count_vectorizer.get_feature_names(),10)


Topic  0
love, don, know, like, just, got, ll, let, want, come

Topic  1
love, love love, got love, heart, way love, love don, need love, love somebody, love need, love just

Topic  2
like, got, like like, ya, em, love, ain, walk, girl, way

Topic  3
got, ll, come, let, ve, ve got, time, gonna, make, just

Topic  4
got, don, ve, ve got, vibe, got love, got got, somebody, got vibe, love somebody

Topic  5
let, come, don, come come, ya, wanna, let let, dance, make, stop

Topic  6
let, know, come, don know, ya, got, let let, love, come come, let know

Topic  7
just, wanna, ya, want, let, say, ain, ya ya, girl, dat

Topic  8
come, come come, want, wanna, girl, just, ride, know, need, little

Topic  9
ya, ll, ya ya, dat, dat dat, man, want, make, girl, need

Topic  10
ya, just, come, gonna, time, ya ya, dat, ain, don, dat dat

Topic  11
just, want, ya, come, ya ya, let, dat, away, dat dat, say

Topic  12
wanna, just, ll, got, ya, know, let, don wanna, just wanna, jump

Topic  13
want, gonn

In [45]:
display_topics(nmf_cv, count_vectorizer.get_feature_names(),10)


Topic  0
just, little, verse, ain, cause, life, man, just like, night, just little

Topic  1
love, love love, heart, ve, need, true, somebody, love don, sweet, got love

Topic  2
don, stop, don know, don don, don want, don stop, don wanna, love don, cause, don need

Topic  3
like, like like, em, verse, ain, shit, bitch, cause, rock, fuck

Topic  4
got, ve, ve got, vibe, ain, got got, got love, somebody, got vibe, man

Topic  5
know, don know, know know, know don, tell, gotta, cause, know love, let know, really

Topic  6
let, let let, let know, won, night, make, rock, don let, won let, krazy

Topic  7
ll, make, ll ll, kiss, heart, ll make, need, know ll, jump, cause

Topic  8
come, come come, ride, make, train, little, home, come let, ride ride, come make

Topic  9
ya, ya ya, dat, dat dat, man, ain, yo, hook, tryin, shit

Topic  10
want, need, don want, want want, want love, know want, verse, bad, really, want don

Topic  11
wanna, make, don wanna, wanna wanna, really, rock, just wanna

### Time to build my first recommendation system. I probably have to go back and make it better but it's nice just to have a first iteration of me doing the thing

In [46]:
test_lyrics = test.drop(['artist', 'song'], axis = 1, errors = 'ignore').lyrics.tolist()
test_lyrics[:5]

["The blues is knocking on my door\nJust like it did once before\nNothing new, just the same old thing\nYou got me singing the blues again\nThe news is all over the town\nYou got me acting like a town\nNothing new, it's the same old thing\nYou got me singing the blues again\nTelling lies is all you know\nYou have me believing they were so\nNothing new, just the same old thing\nYou got me singing the blues again\nYou build me up, then you let me down\nYou got my head going 'round and 'round\nNothing new, just the same old thing\nYou got me singing the blues again",
 "Wrapped around each other\nTrying so hard to stay warm\nThat first cold winter together\nLying in each other's arms\nWatching those old movies\nFalling in love so desperately\nHoney, I was your hero\nAnd you were my leading lady\n\nWe had it all\nJust like Bogie and Bacall\nStarring in our own late, late show\nSailing away to Key Largo\n\nHere's lookin' at you kid\nMissing all the things we did\nWe can find it once again, I

In [47]:
def get_recommendations(first_article, model, vectorizer, training_vectors):
    new_vec = model.transform(
        vectorizer.transform([first_article]))
    nn = NearestNeighbors(n_neighbors=10, metric='cosine', algorithm='brute')
    nn.fit(training_vectors)
    results = nn.kneighbors(new_vec)
    return results[1][0]

print(get_recommendations(test_lyrics[0], lsa_cv, count_vectorizer, lsa_cv_data))

[11361  1336 10433  2975 11538  9164   383  8699  7124  6706]


In [48]:
def print_recommendations(first_article,recommend_list):
    print(first_article)
    print('\n------\n')
    for resp in recommend_list:
        print('\n --- Result --- \n')
        print(song_list[resp])
        
rec_list = get_recommendations(song_list[0], lsa_cv, count_vectorizer, lsa_cv_data)

print_recommendations(test_lyrics[0],rec_list)

The blues is knocking on my door
Just like it did once before
Nothing new, just the same old thing
You got me singing the blues again
The news is all over the town
You got me acting like a town
Nothing new, it's the same old thing
You got me singing the blues again
Telling lies is all you know
You have me believing they were so
Nothing new, just the same old thing
You got me singing the blues again
You build me up, then you let me down
You got my head going 'round and 'round
Nothing new, just the same old thing
You got me singing the blues again

------


 --- Result --- 

When push comes to a shoving
I'd rather make some lovin'
And without being sneaky
Or sounding to you freaky
I get high
I get high
What ya say
I get high on you
Naturally my temperature is rising
I hope you're realizing
To know you is my pleasure
And you're my only treasure
I get high on you
I get high on you
I get high on you
Sing it
Even when I'm sleeping
Your body seems to creep in
And slowly you awake me
Wow Mothe